In [35]:
import pandas as pd
import numpy as np
import re
import csv

from gensim.models import Word2Vec
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

In [36]:
df = pd.read_csv('train.csv',error_bad_lines = False, encoding = "ISO-8859-1")

with open('acronym_dict.csv', 'r') as csv_file:
    reader = csv.reader(csv_file)
    acronym = dict(reader)

with open('emoticon.csv', 'r') as csv_file:
    reader = csv.reader(csv_file)
    emot = dict(reader)

In [37]:
ds = df.values

In [38]:
for i in range (0,ds.shape[0]) :
    k = ds[i][2].split()
    
    for j in range (len(k)) :
        if 'https' in k[j] or 'http' in k[j] or 'fttp' in k[j] :
            k[j] = '|H|'
            
        if len(k[j]) > 1 and k[j][0] == '@' and k[j][1].isalpha() :
            k[j] = '|N|'
            
        if len(k[j]) > 1 and k[j][0] == '#' and k[j][1].isalpha() :
            k[j] = '|T|'
            
        count = 0
       
        ix = 1

        while ix<len(k[j]):

            if(k[j][ix] == k[j][ix-1]):
                count = count + 1    

                if(count > 2):
                    k[j] = k[j][0:ix] + k[j][ix+1:len(k[j])]
                    ix = ix - 1

            else:
                count = 0

            ix+=1
            
        if k[j] in acronym:
            k[j] = acronym[k[j]]
            
        if k[j] in emot:
            k[j] = emot[k[j]]
            
    ds[i][2] = ' '.join(k)
    ds[i][2] = re.sub('[^A-Za-z|]',' ',ds[i][2]).lower().split()
    
sent =[]

for i in range (0,ds.shape[0]) :
    sent.append(ds[i][2])    

In [39]:
ndim = 220
tweet_w2v = Word2Vec(sent, size=ndim, window=10, min_count=5, workers=4)

In [40]:
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x for x in sent])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

In [42]:
emb = []

for i in range(len(sent)):
    
    vec = np.zeros(ndim)
    count = 0
    
    for word in sent[i]:
        try:
            vec += tweet_w2v[word] * tfidf[word]  
            count += 1.
        except : 
            continue
            
    if count != 0:
        vec /= count

    emb.append(vec)

/home/shubham/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [46]:
# emb = []
# for i in range (0, len(sent)) :
#     temp = []
#     for word in sent[i] :
#         try :
#             temp.append(tweet_w2v[word])
#         except :
#             pass
    
#     emb.append(temp)
    
label = []

for i in range (ds.shape[0]) :
    label.append(ds[i][1])
    
train_vecs_w2v = scale(np.array(emb)) #np.array(emb)

x_train, x_test, y_train, y_test = train_test_split(train_vecs_w2v, label, test_size=0.2)

In [47]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(ndim,)))
model.add(Dense(64, activation='tanh'))
model.add(Dropout(0.25))
model.add(Dense(32, activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adagrad', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 128)               28288     
_________________________________________________________________
dense_18 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_10 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 33        
Total params: 38,657
Trainable params: 38,657
Non-trainable params: 0
_________________________________________________________________


In [48]:
hist = model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test))

Train on 79991 samples, validate on 19998 samples
Epoch 1/20
79991/79991 [==============================] - 5s 58us/step - loss: 0.5476 - acc: 0.7179 - val_loss: 0.5239 - val_acc: 0.7347
Epoch 2/20
79991/79991 [==============================] - 4s 56us/step - loss: 0.5264 - acc: 0.7339 - val_loss: 0.5184 - val_acc: 0.7392
Epoch 3/20
79991/79991 [==============================] - 5s 65us/step - loss: 0.5201 - acc: 0.7391 - val_loss: 0.5152 - val_acc: 0.7394
Epoch 4/20
79991/79991 [==============================] - 8s 97us/step - loss: 0.5142 - acc: 0.7411 - val_loss: 0.5138 - val_acc: 0.7421
Epoch 5/20
79991/79991 [==============================] - 6s 70us/step - loss: 0.5109 - acc: 0.7452 - val_loss: 0.5116 - val_acc: 0.7430
Epoch 6/20
79991/79991 [==============================] - 6s 79us/step - loss: 0.5086 - acc: 0.7454 - val_loss: 0.5104 - val_acc: 0.7430
Epoch 7/20
79991/79991 [==============================] - 5s 62us/step - loss: 0.5050 - acc: 0.7486 - val_loss: 0.5107 - val_acc

In [32]:
w = model.predict(x_test)

for i in range (len(w)) :
    if w[i] < 0.5 :
        w[i] = 0
    
    else :
        w[i] = 1  
        
w = w.reshape(w.shape[0],)
r = np.array(y_test)

In [33]:
accuracy_score(r, w), precision_recall_fscore_support(r, w, average='binary')    # precision recall fbeta_score

(0.74752475247524752,
 (0.76987012987012982, 0.78808827439510765, 0.77887268427276302, None))